# Card Classifier

---

#### 1. Preparing the data

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import sys

In [3]:
print('System:', sys.version)
print('PyTorch:', torch.__version__)
print('Torchvision:', torchvision.__version__)
print('Numpy:', np.__version__)
print('Pandas:', pd.__version__)

System: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]
PyTorch: 2.7.1
Torchvision: 0.22.1
Numpy: 2.3.1
Pandas: 2.2.3


In [12]:
class CardDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        return self.data[idx]

    @property
    def classes(self):
        return self.data.classes

In [13]:
data_dir = '../data/train'

In [14]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

In [40]:
dataset = CardDataset(data_dir, transform)

In [41]:
len(dataset)

7624

In [42]:
img, lbl = dataset[1000]

In [47]:
img.shape, lbl

(torch.Size([3, 128, 128]), 6)

In [48]:
data_dir = '../data/train'
target_to_class = {v: k for k, v in ImageFolder(data_dir).class_to_idx.items()}
#print(target_to_class)

In [49]:
dataloader = DataLoader(dataset, batch_size=34, shuffle=True)

In [53]:
for img, lbl in dataloader:
    break

In [55]:
img.shape

torch.Size([34, 3, 128, 128])

In [54]:
lbl

tensor([23, 22,  8,  8, 10, 30, 41, 10, 26,  6,  6, 37, 10,  2, 16, 36, 14, 18,
        38, 45,  9, 50, 24,  9, 48, 16, 50, 41, 18, 18,  7, 27, 41,  6])

---

#### 2. Creating the model